In [3]:
import numpy as np
import pdb

In [9]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  pdb.set_trace()
  Xy = np.concatenate((X, y.T, predictions.T), axis=1)
  district_scores = np.array([])
  for key in np.unique(groups):
    pdb.set_trace()
    district_scores[len(district_scores)] = (
      np.sum(
        (Xy[np.where(Xy[0] == key)][:,-2] -
         Xy[np.where(Xy[0] == key)][:,-1])/
        Xy[np.where(Xy[0] == key)][:,-2]
      ) / num_timeslots
    )
  return np.sum(district_scores) / num_districts

def mape_scorer(estimator, X, y):
  estimator.fit(X)
  predictions = estimator.predict(X)
  return mape(X, predictions, y)

Testing MAPE

In [10]:
from sklearn.linear_model import LogisticRegression

est = LogisticRegression()
X = np.array([[1, 1], [1, 2], [2, 3], [2, 4]])
predictions = np.array([1, 2])
y = np.array([1, 2])

# Should return 0
mape(X, predictions, y)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
features_list = ['']